In [ ]:
import pandas as pd
import numpy as np
import os
from openai import OpenAI
from dotenv import load_dotenv

In [2]:
load_dotenv()
%matplotlib inline
client = OpenAI(api_key = os.getenv("OPENAI_API_KEY"))

In [3]:
df = pd.read_csv('features_full_final.csv')

In [4]:
df.columns

Index(['mbid', 'title', 'artist', 'genre', 'year', 'duration_ms',
       'high_danceability_value', 'high_danceability_probability',
       'high_gender_value', 'high_gender_probability',
       'high_genre_dortmund_value', 'high_genre_dortmund_probability',
       'high_genre_electronic_value', 'high_genre_electronic_probability',
       'high_genre_rosamerica_value', 'high_genre_rosamerica_probability',
       'high_genre_tzanetakis_value', 'high_genre_tzanetakis_probability',
       'high_ismir04_rhythm_value', 'high_ismir04_rhythm_probability',
       'high_mood_acoustic_value', 'high_mood_acoustic_probability',
       'high_mood_aggressive_value', 'high_mood_aggressive_probability',
       'high_mood_electronic_value', 'high_mood_electronic_probability',
       'high_mood_happy_value', 'high_mood_happy_probability',
       'high_mood_party_value', 'high_mood_party_probability',
       'high_mood_relaxed_value', 'high_mood_relaxed_probability',
       'high_mood_sad_value', 'high_m

In [5]:
def filtrar_features(df:pd.DataFrame):
    ritmo_energia = [
        'low_bpm', 'low_onset_rate', 'low_danceability',
        'low_dynamic_complexity', 'low_average_loudness'
    ]
    tonalidad = [
        'low_key_strength', 'low_chords_changes_rate', 'low_chords_number_rate'
    ]
    mfccs = [f'low_mfcc_mean_{i}' for i in range(13)]
    moods = [
        'high_mood_acoustic_probability',
        'high_mood_aggressive_probability',
        'high_mood_electronic_probability',
        'high_mood_happy_probability',
        'high_mood_party_probability',
        'high_mood_relaxed_probability',
        'high_mood_sad_probability'
    ]
    columnas = ['genre'] + ritmo_energia + tonalidad + mfccs + moods
    return df[columnas].copy()

In [6]:
datos = filtrar_features(df)

In [7]:
datos.shape

(111956, 29)

In [8]:
datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111956 entries, 0 to 111955
Data columns (total 29 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   genre                             111956 non-null  object 
 1   low_bpm                           111955 non-null  float64
 2   low_onset_rate                    111955 non-null  float64
 3   low_danceability                  111955 non-null  float64
 4   low_dynamic_complexity            111956 non-null  float64
 5   low_average_loudness              111956 non-null  float64
 6   low_key_strength                  111899 non-null  float64
 7   low_chords_changes_rate           111955 non-null  float64
 8   low_chords_number_rate            111955 non-null  float64
 9   low_mfcc_mean_0                   111956 non-null  float64
 10  low_mfcc_mean_1                   111956 non-null  float64
 11  low_mfcc_mean_2                   111956 non-null  f

In [9]:
datos.dropna(inplace=True)
datos.shape

(111752, 29)

In [10]:
def promedio_genero(df_filtrado, genero):
    return df_filtrado[df_filtrado['genre'] == genero].mean(numeric_only=True)

In [11]:
promedio_genero(datos, 'rock')

low_bpm                             126.046423
low_onset_rate                        3.222548
low_danceability                      1.117270
low_dynamic_complexity                3.838504
low_average_loudness                  0.799274
low_key_strength                      0.706085
low_chords_changes_rate               0.062181
low_chords_number_rate                0.002559
low_mfcc_mean_0                    -669.521063
low_mfcc_mean_1                     131.665669
low_mfcc_mean_2                      -2.986507
low_mfcc_mean_3                      25.324911
low_mfcc_mean_4                       4.910339
low_mfcc_mean_5                       5.981352
low_mfcc_mean_6                       0.229045
low_mfcc_mean_7                       3.361125
low_mfcc_mean_8                      -0.777716
low_mfcc_mean_9                       1.304612
low_mfcc_mean_10                     -1.822891
low_mfcc_mean_11                     -0.659284
low_mfcc_mean_12                     -1.810811
high_mood_aco

In [12]:
def comparar_cancion_con_promedio(nueva_cancion:pd.Series, promedio_genero:pd.Series) -> dict:
    nueva_cancion.drop(['genre'], inplace=True)
    diferencias = {}
    for feature in nueva_cancion.index:
        valor_actual = nueva_cancion[feature]
        valor_referencia = promedio_genero[feature]
        diferencias[feature] = round(float(valor_actual-valor_referencia),3)
    return diferencias

In [13]:
nueva_cancion = filtrar_features(df.iloc[123])

In [14]:
genero = nueva_cancion['genre']
genero

'rock'

In [15]:
promedios = promedio_genero(datos, nueva_cancion['genre'])

In [16]:
diferencias = comparar_cancion_con_promedio(nueva_cancion,promedios)

In [17]:
diferencias

{'low_bpm': -14.189,
 'low_onset_rate': -0.152,
 'low_danceability': 0.064,
 'low_dynamic_complexity': 0.085,
 'low_average_loudness': 0.124,
 'low_key_strength': 0.042,
 'low_chords_changes_rate': 0.004,
 'low_chords_number_rate': -0.0,
 'low_mfcc_mean_0': 1.02,
 'low_mfcc_mean_1': -24.764,
 'low_mfcc_mean_2': 23.814,
 'low_mfcc_mean_3': 7.773,
 'low_mfcc_mean_4': 8.52,
 'low_mfcc_mean_5': 1.696,
 'low_mfcc_mean_6': 3.59,
 'low_mfcc_mean_7': -3.046,
 'low_mfcc_mean_8': 0.01,
 'low_mfcc_mean_9': 0.331,
 'low_mfcc_mean_10': -5.199,
 'low_mfcc_mean_11': -0.682,
 'low_mfcc_mean_12': -3.176,
 'high_mood_acoustic_probability': 0.002,
 'high_mood_aggressive_probability': 0.108,
 'high_mood_electronic_probability': -0.177,
 'high_mood_happy_probability': -0.205,
 'high_mood_party_probability': -0.031,
 'high_mood_relaxed_probability': -0.252,
 'high_mood_sad_probability': -0.11}

In [34]:
def generar_prompt(genero, diferencias, nueva_cancion, promedio_genero, porcentaje_umbral=0.05):
    sugerencias = []

    for feature, diferencia in diferencias.items():
        val_actual = nueva_cancion[feature]
        val_prom = promedio_genero[feature]

        umbral = abs(val_prom * porcentaje_umbral) if val_prom != 0 else porcentaje_umbral

        if abs(diferencia) > umbral:
            if diferencia < 0:
                sugerencias.append(
                    f"- '{feature}': el valor actual es {val_actual:.2f}, mientras que el promedio del género es {val_prom:.2f}. Está por debajo del promedio; podrías intentar incrementarlo.")
            else:
                sugerencias.append(
                    f"- '{feature}': el valor actual es {val_actual:.2f}, mientras que el promedio del género es {val_prom:.2f}. Está por encima del promedio; podrías ajustarlo.")

    if not sugerencias:
        sugerencias.append("Los valores están bastante alineados con el promedio del género.")

    prompt = f"Tengo una canción del género {genero}. Estas son las diferencias respecto al promedio de las canciones más populares del mismo género:\n\n"
    prompt += "\n".join(sugerencias)
    prompt += "\n\nDame recomendaciones musicales para mejorar esta canción"
    prompt += "\nLos nombres de las features vienen en inglés, pero la respuesta debe ser todo en castellano."

    return prompt

In [35]:
promt = generar_prompt(genero, diferencias, nueva_cancion, promedios)

In [36]:
print(promt)

Tengo una canción del género rock. Estas son las diferencias respecto al promedio de las canciones más populares del mismo género:

- 'low_bpm': el valor actual es 111.86, mientras que el promedio del género es 126.05. Está por debajo del promedio; podrías intentar incrementarlo.
- 'low_danceability': el valor actual es 1.18, mientras que el promedio del género es 1.12. Está por encima del promedio; podrías ajustarlo.
- 'low_average_loudness': el valor actual es 0.92, mientras que el promedio del género es 0.80. Está por encima del promedio; podrías ajustarlo.
- 'low_key_strength': el valor actual es 0.75, mientras que el promedio del género es 0.71. Está por encima del promedio; podrías ajustarlo.
- 'low_chords_changes_rate': el valor actual es 0.07, mientras que el promedio del género es 0.06. Está por encima del promedio; podrías ajustarlo.
- 'low_mfcc_mean_1': el valor actual es 106.90, mientras que el promedio del género es 131.67. Está por debajo del promedio; podrías intentar in

In [31]:
def obtener_recomendacion(prompt):
    response = client.chat.completions.create(
        model='gpt-4.1-mini',
        messages=[
            {"role": "system", "content": "Eres una app de música que brinda consejos claros y útiles sobre cómo mejorar una canción según sus características."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.7,
        max_tokens=1000
    )
    return response.choices[0].message.content

In [37]:
import time

inicio = time.time()
recomendacion = obtener_recomendacion(promt)
fin = time.time()

print(f"Tiempo de ejecución: {fin - inicio:.2f} segundos")

Tiempo de ejecución: 6.60 segundos


In [38]:
print(recomendacion)

Para mejorar tu canción de rock y acercarla más al perfil de las canciones más populares del género, aquí tienes algunas recomendaciones basadas en las diferencias que mencionas:

1. **Incrementar el tempo (bpm)**: Actualmente está en 111.86 bpm, por debajo del promedio que es 126.05 bpm. Subir un poco el tempo puede darle más energía y dinamismo, típico del rock popular.

2. **Ajustar la danceabilidad**: Tu valor es ligeramente superior al promedio. Puedes hacer pequeños ajustes para que la canción mantenga un ritmo que invite a moverse pero sin perder la esencia del rock.

3. **Reducir la sonoridad promedio**: El valor de loudness está un poco alto; bajar ligeramente el volumen promedio o controlar mejor la dinámica puede hacer que la canción suene más equilibrada y profesional.

4. **Ajustar la fuerza tonal (key strength)**: Está un poco por encima, podrías hacer que la tonalidad sea un poco más sutil para evitar que suene demasiado rígida.

5. **Reducir la tasa de cambios de acorde